# Loop 5 LB Feedback Analysis

## Key Observations
- LB Score: 70.6761 (matches CV exactly - deterministic problem)
- Target: 68.9228 (gap of 1.75 points = 2.5%)
- **CRITICAL**: Target is BELOW current LB #1 (71.19)

## What We've Tried
1. Pre-optimized submission from SmartManoj (70.74)
2. bbox3 C++ optimizer (0 improvement)
3. Ensemble from 17 sources (70.676)
4. sa_v1_parallel optimizer (0 improvement)
5. Backward iteration/backpacking (0 improvement)

## What We Haven't Tried Yet
1. **Corner extraction** from chistyakov kernel - extract smaller layouts from larger ones
2. **Well-aligned initial solutions** from zaburo kernel - structured grid packing
3. **Generating NEW configurations** from scratch for small N
4. **Different random restarts** - all current solutions may share same local optimum basin

In [ ]:
import pandas as pd
import numpy as np
import os

# Load current best submission
df = pd.read_csv('/home/submission/submission.csv')
print(f'Loaded {len(df)} rows')

In [ ]:
# Calculate per-N scores
def strip(val):
    return float(str(val).replace('s', ''))

TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_side_length(xs, ys, degs):
    all_px, all_py = [], []
    for x, y, deg in zip(xs, ys, degs):
        rad = np.radians(deg)
        c, s = np.cos(rad), np.sin(rad)
        px = TX * c - TY * s + x
        py = TX * s + TY * c + y
        all_px.extend(px)
        all_py.extend(py)
    return max(max(all_px) - min(all_px), max(all_py) - min(all_py))

def score_group(xs, ys, degs):
    n = len(xs)
    side = get_side_length(xs, ys, degs)
    return side * side / n

# Calculate per-N scores
scores = {}
for n in range(1, 201):
    group = df[df['id'].str.startswith(f'{n:03d}_')]
    xs = np.array([strip(x) for x in group['x']])
    ys = np.array([strip(y) for y in group['y']])
    degs = np.array([strip(d) for d in group['deg']])
    scores[n] = score_group(xs, ys, degs)

total = sum(scores.values())
print(f'Total score: {total:.6f}')
print(f'Target: 68.922808')
print(f'Gap: {total - 68.922808:.6f} ({(total - 68.922808) / 68.922808 * 100:.2f}%)')

In [ ]:
# Analyze worst N values
worst = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:30]
print('\nWorst 30 N values (highest contribution):')
print('N\tScore\t% of total')
for n, s in worst:
    print(f'{n}\t{s:.6f}\t{s/total*100:.2f}%')

In [ ]:
# Calculate how much improvement is needed
target = 68.922808
gap = total - target
print(f'\nTotal gap to close: {gap:.6f}')

# If we improved top 10 worst N values by X%, what would we get?
for pct in [10, 20, 30, 40, 50]:
    top10_sum = sum(s for n, s in worst[:10])
    improvement = top10_sum * pct / 100
    new_total = total - improvement
    print(f'Improve top 10 by {pct}%: {new_total:.6f} (gap: {new_total - target:.6f})')

In [ ]:
# Check what sources we have
import glob

csv_files = glob.glob('/home/code/datasets/*.csv')
print(f'\nCSV files in datasets:')
for f in csv_files:
    print(f'  {os.path.basename(f)}')

# Also check experiments
exp_csvs = glob.glob('/home/code/experiments/*/*.csv')
print(f'\nCSV files in experiments:')
for f in exp_csvs[:10]:
    print(f'  {f}')

In [ ]:
# Key insight: We need to GENERATE new solutions, not just optimize existing ones
# The current ensemble is at a strong local optimum

# Approaches to try:
# 1. Corner extraction (chistyakov kernel) - extract smaller layouts from larger ones
# 2. Well-aligned grid packing (zaburo kernel) - structured initial solutions
# 3. Random restart with different initial configurations
# 4. Focus on small N (1-30) where improvements have highest leverage

print('\n=== STRATEGY RECOMMENDATIONS ===')
print('1. Implement corner extraction from chistyakov kernel')
print('2. Try well-aligned grid packing from zaburo kernel')
print('3. Generate random configurations for small N and optimize')
print('4. Use genetic crossover between different configurations')